In [1]:
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import random
import itertools
import pandas as pd

from opyenxes.model.XLog import XLog
from opyenxes.data_in.XUniversalParser import XUniversalParser
from opyenxes.classification.XEventAttributeClassifier import XEventAttributeClassifier

In [2]:
# set up chainer on a single GPU for now
device_id = -1
try:
    cuda.check_cuda_available()
    cuda.get_device(device_id).use()
    import cupy
    print("running on GPU, switching numpy for cupy!")
    xp = cupy
    device_id = 0
except:
    xp = np

In [3]:
bpic_path = "../logs/bpic2011.xes"

with open(bpic_path) as bpic_file:
    eventlog = XUniversalParser().parse(bpic_file)[0]

Unknown extension: http://www.xes-standard.org/meta_time.xesext
Unknown extension: http://www.xes-standard.org/meta_life.xesext
Unknown extension: http://www.xes-standard.org/meta_org.xesext
Unknown extension: http://www.xes-standard.org/meta_concept.xesext
Unknown extension: http://www.xes-standard.org/meta_3TU.xesext
Unknown extension: http://www.xes-standard.org/meta_general.xesext


## Dataset and Iterator Setup

In [ ]:
# collect all attributes
attribute_list = []

# extract column names from any trace, here the first is used
for event in eventlog[0]:
    event_attributes = event.get_attributes()
    
    for attribute in event_attributes:
        attribute_list.append(attribute)
        
attribute_list = set(attribute_list) # remove duplicates
column_names   = ["__case_id"] + list(attribute_list)

log_length    = sum([2+len(t) for t in eventlog])
event_indices = range(0, log_length) # total number of entries in log
eventlog_df   = pd.DataFrame(columns=column_names, index=event_indices)
row_idx       = 0

def set_row_value(df, row, colnames, val):
    for column in colnames:
        df.iloc[row][column] = val

for trace_idx, raw_trace in enumerate(eventlog):
    # insert start-of-sequence marker
    set_row_value(eventlog_df, row_idx, column_names, "<bos>")
    
    for event_idx, event in enumerate(raw_trace):
        event_attributes = event.get_attributes()
        eventlog_df.iloc[row_idx]["__case_id"] = trace_idx
        
        for attribute in event_attributes:
            eventlog_df.iloc[row_idx][attribute] = event_attributes[attribute].get_value()
            
        row_idx += 1
    # finalize trace by inserting end-of-sequence marker    
    set_row_value(eventlog_df, row_idx, column_names, "<eos>")
    row_idx += 1

In [4]:
# use a generator for the generation of every sample
def window_features(traces,windowsize):
    for trace in traces:
        for event_i in range(0, len(trace)-windowsize+1):
            encoded_window = [event_to_int[trace[i]] for i in range(event_i, event_i+windowsize)]
            yield(encoded_window)
            
# extract event names and enrich with beginning and end markers
event_traces = [[ ev.get_attributes()["concept:name"].get_value() for ev in trace ] for trace in bpic2011_log ]
event_traces = [ ['<bos>'] + l + ['<eos>'] for l in event_traces ]

# generate word mappings to IDs
events       = sorted(list(set(itertools.chain.from_iterable(event_traces)))) 
event_to_int = dict((c, i) for i,c in enumerate(events))
int_to_event = dict((i, c) for i,c in enumerate(events))

# shuffle complete traces and create test and training set
random.shuffle(event_traces)
train_traces = event_traces[:int(.8*len(event_traces))]
test_traces  = event_traces[int(.8*len(event_traces)):]

# from these sets, create feature windows for learning
trace_dt = xp.float32
window_size = 5
train_traces = xp.array([ w for w in window_features(train_traces, window_size) ], dtype=trace_dt)
test_traces  = xp.array([ w for w in window_features(test_traces,  window_size) ],  dtype=trace_dt)

#  extract all columns but the last from all rows
train_x = train_traces[:, :4]
# extract only the last column and put each element into an array of its own
train_y = (train_traces[:, 4][:, None]).flatten().astype(xp.int32)

#  extract all columns but the last from all rows
test_x = test_traces[:, :4]
# extract only the last column and put each element into an array of its own
test_y = (test_traces[:, 4][:, None]).flatten().astype(xp.int32)

train_ds = datasets.TupleDataset(train_x, train_y)
test_ds  = datasets.TupleDataset(test_x, test_y)

train_iter = chainer.iterators.SerialIterator(train_ds, 100, repeat=True, shuffle=False)
test_iter  = chainer.iterators.SerialIterator(test_ds,  100, repeat=False, shuffle=False)

## Neural Network Modeling

In [8]:
class HalfLifeModel(Chain):
    def __init__(self, vocab_size, dim_embed=33*3, dim1=400, dim2=400, dim3=200, class_size=None):
        super(HalfLifeModel, self).__init__()
        if class_size is None:
            class_size = vocab_size
        
        # ss = subsequence
        # sq = sequence
        # co = concatenated
        self.sq_embed1    = L.EmbedID(vocab_size, dim_embed)
        self.sq_lstm2     = L.LSTM(dim_embed, dim1, forget_bias_init=0)
        self.sq_lstm3     = L.LSTM(dim1, dim2, forget_bias_init=0)
        
        self.ss_embed1 = L.Linear(vocab_size, dim_embed)
        self.ss_lin2   = L.Linear(dim_embed, dim1)

        self.co_lin1 = L.Linear(dim1+dim2, dim3)
        self.co_lin2 = L.Linear(dim3, class_size)
        
        self.vocab_size = vocab_size
        self.dim_embed  = dim_embed
        self.loss_var = Variable(xp.zeros((), dtype=np.float32))
        self.reset_state()

    def __call__(self, x, train):
        print(x)
        seq_window = x[0]
        ss_vector  = x[1]
        
        x_uni = x_3gram[:,0]
        y  = Variable(x_uni, volatile = not train)
        y  = self.sq_embed1(y)     
        y2 = self.sq_lstm2(y)
        y2 = self.sq_lstm3(y2)        

        y = Variable(sp2, volatile = not train)
        y = self.ss_embed1(y)
        y = self.ss_lin2(y)
        y3 = F.relu(y)
        
        y = concat.concat((y2,y3) )
        y = self.co_lin1(F.dropout(y, train=train))
        y = F.relu(y)
        y = self.co_lin2(F.dropout(y, train=train)) 
        
        return y

    def reset_state(self):
        if self.loss_var is not None:
            self.loss_var.unchain_backward()
            
        self.loss_var = Variable(xp.zeros((), dtype=xp.float32))
        self.sq_lstm2.reset_state()
        self.sq_lstm3.reset_state()
        return
    
gordon = HalfLifeModel(vocab_size=4, class_size=len(events))
model = L.Classifier(gordon, accfun=F.accuracy)
optimizer = optimizers.MomentumSGD().setup(model)

## Linking Iterator And Optimizer together

In [14]:
updater = training.StandardUpdater(train_iter, optimizer, device=device_id)
trainer = training.Trainer(updater, (1, 'epoch'), out='result')

trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.Evaluator(test_iter, model, device=device_id))

In [ ]:
if device_id != -1:
    model.to_gpu()

trainer.run()